In [16]:
import sqlite3 as db
import pandas as pd
import re

In [17]:
conn = db.connect('mydatabase.db')
df_h2 = pd.read_sql_query('select * from "H2_final_subsectors_F"', conn)
df_hp = pd.read_sql_query('select * from "Heat_potential_final_subsectors_F"', conn)
conn.close()

#### 1. Merge common data entries

In [19]:
merged_df = pd.merge(df_h2, df_hp, left_on='Register_identifier', right_on='Register_identifier', how='inner')
merged_df.head()


,Name_x,Register_identifier,Postalcode,Subsector_Name,Stammkapital_x,Grundkapital_x,Hafteinlage_x,Latitude_x,Longitude_x,Address_x,...,level_1_r_Tj,level_2_r_Tj,level_3_r_Tj,Capital_y,level_1_Tj_proportional,level_2_Tj_proportional,level_3_Tj_proportional,level_1_r_Tj_proportional,level_2_r_Tj_proportional,level_3_r_Tj_proportional
0,AGC Glass Osterweddingen GmbH,HRB_7259,39171,Non-metallic minerals,1100000.0,0.0,0.0,52.0594816,11.5683208,"Appendorfer Weg 5, 39171 Sülzetal, Germany",...,331948593,115274154,83814366,1100000.0,1.209091e+09,9.535269e+08,916420173.0,3.319486e+08,1.152742e+08,8.381437e+07
1,ArcelorMittal Bremen GmbH,HRB_15474,28237,Metal Processing,270000000.0,0.0,0.0,53.14184495,8.69176220707154,"Hauptverwaltung, 30, Carl-Benz-Straße, Industr...",...,1885154742,1518450228,1108092726,270000000.0,2.597804e+09,2.221140e+09,180006903.0,1.885155e+09,1.518450e+09,1.108093e+09
2,ArcelorMittal Bremen GmbH,HRB_15474,28237,"Steel, primary",270000000.0,0.0,0.0,53.14184495,8.69176220707154,"Hauptverwaltung, 30, Carl-Benz-Straße, Industr...",...,1885154742,1518450228,1108092726,270000000.0,2.597804e+09,2.221140e+09,180006903.0,1.885155e+09,1.518450e+09,1.108093e+09
3,ArcelorMittal Duisburg GmbH,HRB_22003,47137,Metal Processing,225000.0,0.0,0.0,51.46088955,6.74905971145791,"107, Vohwinkelstraße, Untermeiderich, Meideric...",...,4453191,387711,3100239,225000.0,3.793958e+07,3.701512e+07,35768565.0,4.453191e+06,3.877110e+05,3.100239e+06
4,ArcelorMittal Eisenhüttenstadt GmbH,HRB_3883,15890,Metal Processing,178000000.0,0.0,0.0,52.1547005,14.6345007,"Werkstraße, Eisenhüttenstadt, Oder-Spree, Bran...",...,27544743,2243052,16596387,178000000.0,6.468347e+07,5.915073e+07,5278662.0,2.754474e+07,2.243052e+06,1.659639e+07


In [20]:
column_names_list = list(merged_df.columns)
print(column_names_list)


['Name_x', 'Register_identifier', 'Postalcode', 'Subsector_Name', 'Stammkapital_x', 'Grundkapital_x', 'Hafteinlage_x', 'Latitude_x', 'Longitude_x', 'Address_x', 'H2-Potential in TWh', 'Capital_x', 'Distributed H2-Potential in TWh', 'Name_y', 'Eurostat_Name', 'Stammkapital_y', 'Grundkapital_y', 'Hafteinlage_y', 'Latitude_y', 'Longitude_y', 'Address_y', 'PostalCode', 'level_1_Tj', 'level_2_Tj', 'level_3_Tj', 'level_1_r_Tj', 'level_2_r_Tj', 'level_3_r_Tj', 'Capital_y', 'level_1_Tj_proportional', 'level_2_Tj_proportional', 'level_3_Tj_proportional', 'level_1_r_Tj_proportional', 'level_2_r_Tj_proportional', 'level_3_r_Tj_proportional']


In [21]:
merged_df = merged_df.drop(columns=['Stammkapital_y','Grundkapital_y','Hafteinlage_y','Name_y', 'Capital_y', 
                                              'Latitude_y', 'Longitude_y', 'Address_y','Postalcode'])
df_f = merged_df.rename(columns={'Name_x': 'Name', 'Address_x': 'Address',
                                 'Stammkapital_x': 'Stammkapital','Grundkapital_x':'Grundkapital','Hafteinlage_x':'Hafteinlage',
                                 'PostalCode': 'Postal_Code',
                                  'Latitude_x':'Latitude','Longitude_x':'Longitude','Capital_x':'Capital'})


In [22]:
olumn_names_list = list(df_f.columns)
print(olumn_names_list)

['Name', 'Register_identifier', 'Subsector_Name', 'Stammkapital', 'Grundkapital', 'Hafteinlage', 'Latitude', 'Longitude', 'Address', 'H2-Potential in TWh', 'Capital', 'Distributed H2-Potential in TWh', 'Eurostat_Name', 'Postal_Code', 'level_1_Tj', 'level_2_Tj', 'level_3_Tj', 'level_1_r_Tj', 'level_2_r_Tj', 'level_3_r_Tj', 'level_1_Tj_proportional', 'level_2_Tj_proportional', 'level_3_Tj_proportional', 'level_1_r_Tj_proportional', 'level_2_r_Tj_proportional', 'level_3_r_Tj_proportional']


#### 2. Add the rest of entries which are not coincidential

In [23]:
df_h2 = df_h2.rename(columns={'Postalcode': 'Postal_Code'})
df_hp = df_hp.rename(columns={'PostalCode': 'Postal_Code'})

In [24]:
df_filtered_h2 = df_h2[~df_h2['Name'].isin(df_f['Name'])]
df_filtered_h2.head()

,Name,Register_identifier,Postal_Code,Subsector_Name,Stammkapital,Grundkapital,Hafteinlage,Latitude,Longitude,Address,H2-Potential in TWh,Capital,Distributed H2-Potential in TWh
7,ArcelorMittal Hamburg GmbH,HRB_57591,21129,Metal Processing,10000000.0,0.0,0.0,53.52481965,9.89989525,"Verwaltungsgebäude, 33, Dradenaustraße, Walter...",0.3401,10000000.0,0.3401
8,Ardagh Glass GmbH,HRB_30704,31582,Non-metallic minerals,37322600.0,0.0,0.0,52.6594201,9.2115158,"132, Große Drakenburger Straße, Stadtteil Nien...",0.1507,37322600.0,0.1507
9,B.E.S. Brandenburger Elektrostahlwerke GmbH,HRB_4239,14770,Metal Processing,12800000.0,0.0,0.0,52.41021475,12.4999016004932,"40, Woltersdorfer Straße, Quenzsiedlung, Brand...",0.5210,12800000.0,0.5210
21,CEMEX Central Europe GmbH,HRB_15702,15562,Mineral Processing,255646500.0,0.0,0.0,52.4889197,13.8096282,"Frankfurter Ch, 15562 Rüdersdorf bei Berlin, G...",1.9119,255646500.0,0.3134
22,CEMEX Deutschland AG,HRB_15672,15562,Mineral Processing,0.0,384000000.0,0.0,52.4889197,13.8096282,"Frankfurter Ch, 15562 Rüdersdorf bei Berlin, G...",1.9119,384000000.0,0.4708


In [25]:
df_filtered_hp = df_hp[~df_hp['Name'].isin(df_f['Name'])]
df_filtered_hp.head()

,Name,Register_identifier,Eurostat_Name,Stammkapital,Grundkapital,Hafteinlage,Latitude,Longitude,Address,Postal_Code,...,level_1_r_Tj,level_2_r_Tj,level_3_r_Tj,Capital,level_1_Tj_proportional,level_2_Tj_proportional,level_3_Tj_proportional,level_1_r_Tj_proportional,level_2_r_Tj_proportional,level_3_r_Tj_proportional
9,BGH Edelstahl Freital GmbH,HRB_319,Iron and steel,30000000.0,0.0,0.0,50.9939315,13.6399066,"Verwaltung, 1, Am Stahlwerk, Deuben, Freital, ...",01705,...,205254,1715985,1263114,30000000.0,1.301694e+07,1.263884e+07,1.212999e+07,2.052540e+05,1.715985e+06,1.263114e+06
12,Baron-Well GmbH,HRB_200649,Paper and printing,40000.0,0.0,0.0,51.91023635,10.4849676479433,"44, Wolfenbütteler Straße, Unteroker, Oker, Go...",38642,...,7748565,2043045,1380495,40000.0,8.623500e+04,2.641095e+06,1.946385e+06,7.748565e+06,2.043045e+06,1.380495e+06
25,Dow Olefinverbund GmbH,HRB_214698,Chemical industry,236580.0,0.0,0.0,51.3943584,11.9743946,"Dow Port 1, 13, Straße B, Korbetha, Schkopau, ...",06258,...,39462525,10069425,5801625,236580.0,2.307954e+06,7.490246e+06,5.226761e+05,1.995883e+07,5.092756e+06,2.934274e+06
26,Dow Pipeline Gesellschaft mbH & Co. KG,HRA_12735,Chemical industry,0.0,0.0,2630000.0,51.2015074,12.3812515,"Werkstraße, Böhlen, Landkreis Leipzig, Sachsen...",04564,...,30597294195,7807300515,449831304,2630000.0,3.215831e+09,9.000674e+08,5.638304e+09,3.059729e+10,7.807301e+09,4.498313e+08
29,ESF Elbe-Stahlwerke Feralpi GmbH,HRB_5452,Iron and steel,11000000.0,0.0,0.0,51.3135865,13.2820426,"Gröbaer Straße, Neuweida, Riesa, Meißen, Sachs...",01591,...,148239,12393225,912249,11000000.0,9.401126e+07,9.128048e+07,8.760550e+07,1.482390e+05,1.239322e+07,9.122490e+05


#### 3. Format and concatenate

In [26]:
df_h2subset = df_filtered_h2
df_hpsubset = df_filtered_hp

new_columns = [
    'Eurostat_Name', 'level_1_Tj', 'level_2_Tj', 'level_3_Tj', 
    'level_1_r_Tj', 'level_2_r_Tj', 'level_3_r_Tj', 
    'Distributed level_1_Tj', 'Distributed level_2_Tj', 'Distributed level_3_Tj', 
    'Distributed level_1_r_Tj', 'Distributed level_2_r_Tj', 'Distributed level_3_r_Tj'
]
df_h2subset.loc[:, new_columns] = '-'

new_columns2 = [
    'Subsector_Name', 'H2-Potential in TWh', 'Distributed H2-Potential in TWh'
]
df_hpsubset.loc[:,new_columns2] = '-'

In [27]:
df_f = df_f.rename(columns={'level_1_Tj_proportional': 'Distributed level_1_Tj', 'level_2_Tj_proportional': 'Distributed level_2_Tj', 
                            'level_3_Tj_proportional': 'Distributed level_3_Tj',
                            'level_1_r_Tj_proportional': 'Distributed level_1_r_Tj', 'level_2_r_Tj_proportional': 'Distributed level_2_r_Tj', 
                            'level_3_r_Tj_proportional': 'Distributed level_3_r_Tj'})

df_hpsubset = df_hpsubset.rename(columns={'level_1_Tj_proportional': 'Distributed level_1_Tj', 'level_2_Tj_proportional': 'Distributed level_2_Tj', 
                            'level_3_Tj_proportional': 'Distributed level_3_Tj',
                            'level_1_r_Tj_proportional': 'Distributed level_1_r_Tj', 'level_2_r_Tj_proportional': 'Distributed level_2_r_Tj', 
                            'level_3_r_Tj_proportional': 'Distributed level_3_r_Tj'})

df_FINAL = pd.concat([df_f, df_h2subset,df_hpsubset])

#### 4. Set the order of the columns

In [28]:
column_list = list(df_FINAL.columns)
print(column_list)

['Name', 'Register_identifier', 'Subsector_Name', 'Stammkapital', 'Grundkapital', 'Hafteinlage', 'Latitude', 'Longitude', 'Address', 'H2-Potential in TWh', 'Capital', 'Distributed H2-Potential in TWh', 'Eurostat_Name', 'Postal_Code', 'level_1_Tj', 'level_2_Tj', 'level_3_Tj', 'level_1_r_Tj', 'level_2_r_Tj', 'level_3_r_Tj', 'Distributed level_1_Tj', 'Distributed level_2_Tj', 'Distributed level_3_Tj', 'Distributed level_1_r_Tj', 'Distributed level_2_r_Tj', 'Distributed level_3_r_Tj']


In [29]:
# Desired column order
desired_order = ['Name', 'Register_identifier', 'Address', 'Postal_Code','Latitude', 'Longitude','Subsector_Name','Stammkapital', 'Grundkapital', 'Hafteinlage','Capital', 'H2-Potential in TWh','Distributed H2-Potential in TWh','Eurostat_Name', 'level_1_Tj', 'level_2_Tj', 'level_3_Tj', 'level_1_r_Tj', 'level_2_r_Tj', 'level_3_r_Tj', 'Distributed level_1_Tj', 'Distributed level_2_Tj', 'Distributed level_3_Tj', 'Distributed level_1_r_Tj', 'Distributed level_2_r_Tj', 'Distributed level_3_r_Tj']

# Reorder the DataFrame columns
df_FINAL = df_FINAL[desired_order]

#### 5. Save in sqlite database

In [31]:
conn = db.connect('Master_Thesis_Maria.db')

# Save the DataFrame to a table in the SQLite database
#df_FINAL.to_sql('Final_dataset_subsectors', conn, if_exists='replace', index=False)
df_FINAL.to_sql('Energy_intensive_companies_dataset', conn, if_exists='replace', index=False)

# Close the connection
conn.close()